<a href="https://colab.research.google.com/github/AfricArxiv/hub-and-search-portal/blob/master/osf_preprints_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import json
import uuid

preprintsApiUrl = "https://api.osf.io/v2/providers/preprints/africarxiv/preprints/?page[size]=100"
preprintsRequest = requests.get( url = preprintsApiUrl )
preprintsRequestContent = json.loads( preprintsRequest.content )

keys_list = [ 'title', 'date', 'description', 'link', 'doi', 'authors', 'subjects', 'tags' ]
values_list = []

preprintMeta = []
preprintsMeta = []

subjects_list = []
tags_list = []

author = []
preprintAuthors = []
authors = []


for preprint in preprintsRequestContent['data']:

  authorApiUrl = preprint['relationships']['contributors']['links']['related']['href']
  authorRequest = requests.get( url = authorApiUrl )
  authorRequestContent = json.loads( authorRequest.content )

  for preprintAuthor in authorRequestContent['data']:

    author.append( preprintAuthor['embeds']['users']['data']['attributes']['full_name'] )
    author.append( preprintAuthor['embeds']['users']['data']['links']['html'] )

    preprintAuthors.append( author )
    author = []

  for subject in preprint['attributes']['subjects']:
    for subj in subject:
      subjects_list.append( subj['text'] )

  for tag in preprint['attributes']['tags']:
    tags_list.append( tag )

  preprintMeta.append( preprint['attributes']['title'] )
  preprintMeta.append( preprint['attributes']['date_published'] )
  preprintMeta.append( preprint['attributes']['description'] )
  preprintMeta.append( preprint['links']['html'] )
  preprintMeta.append( preprint['links']['preprint_doi'] )

  preprintMeta.append( preprintAuthors )
  preprintAuthors = []

  preprintMeta.append( subjects_list )
  subjects_list = []

  preprintMeta.append( tags_list )
  tags_list = []
  
  preprintsMeta.append( preprintMeta )
  preprintMeta = []
    

In [0]:
preprintsMetaJson = json.dumps( [dict( zip( keys_list, row ) ) for row in preprintsMeta], indent=1 )
prepare_data = json.loads( preprintsMetaJson )
print(prepare_data)

In [0]:
authors_keys = [ 'label', 'element type', 'link' ]
authors_attributes = []
author_data = []
authors_id_list = []

for node in prepare_data:
  for node_authors in node['authors']:
    author_data.append( node_authors[0] )
    author_data.append( 'Author' )
    author_data.append( node_authors[1] )

    authors_unique_id = uuid.uuid4()
    authors_id_list.append( 'elem-' + str( authors_unique_id.time_low ) )

    authors_attributes.append( author_data )
    author_data = []

authors_attributes = json.loads( json.dumps( [dict( zip( authors_keys, row ) ) for row in authors_attributes], indent=1 ) )

authors_format = {}
authors_final_format = []

i = 0

for authors_temp in authors_attributes:
  authors_format['_id'] = authors_id_list[i]
  authors_format['attributes'] = authors_temp

  i += 1

  authors_final_format.append( authors_format )
  authors_format = {}

for element in authors_final_format:
  authors_format.update( element )

In [0]:
subjects_keys = [ 'label', 'element_type' ]
subjects_attributes = []
subject_data = []
subjects_id_list = []

for node in prepare_data:
  for node_subjects in node['subjects']:
    subject_data.append( node_subjects )
    subject_data.append( 'Subject' )

    subjects_unique_id = uuid.uuid4()
    subjects_id_list.append( 'elem-' + str( subjects_unique_id.time_low ) )

    subjects_attributes.append( subject_data )
    subject_data = []

subjects_attributes = json.loads( json.dumps( [dict( zip( subjects_keys, row ) ) for row in subjects_attributes], indent=1 ) )
  
subjects_format = {}
subjects_final_format = []

i = 0

for subjects_temp in subjects_attributes:
  subjects_format['_id'] = subjects_id_list[i]
  subjects_format['attributes'] = subjects_temp

  i += 1

  subjects_final_format.append( subjects_format )
  subjects_format = {}

  subjects_final_format.append( subjects_format )

for element in subjects_final_format:
  subjects_format.update( element )


In [0]:
preprints_keys = [ 'label', 'element type', 'description', 'link', 'doi', 'date', 'tags' ]
preprints_attributes = []
preprint_data = []
preprints_id_list = []

for node in prepare_data:
  preprint_data.append( node['title'] )
  preprint_data.append( 'Preprint' )
  preprint_data.append( node['description'] )
  preprint_data.append( node['link'] )
  preprint_data.append( node['doi'] )
  preprint_data.append( node['date'] )
  preprint_data.append( node['tags'] )

  preprint_unique_id = uuid.uuid4()
  preprints_id_list.append( 'elem-' + str( preprint_unique_id.time_low ) )

  preprints_attributes.append( preprint_data )
  preprint_data = []

preprints_attributes = json.loads( json.dumps( [dict( zip( preprints_keys, row ) ) for row in preprints_attributes], indent=1 ) )

preprints_format = {}
preprints_final_format = []

i = 0

for preprints_temp in preprints_attributes:
  preprints_format['_id'] = preprints_id_list[i]
  preprints_format['attributes'] = preprints_temp

  i = 1

  preprints_final_format.append( preprints_format )
  preprints_format = {}


In [0]:
for element in preprints_final_format:
  print(type(element))

In [0]:
final_list = []
final_dict = {}
final_output = {}

final_list.append( preprints_final_format )
final_list.append( authors_final_format )
final_list.append( subjects_final_format )

for element in final_list:
  final_output.update( element )

print(final_list)
